In [1]:
import argparse, os
import cv2
import numpy as np
import imageio
import time

In [3]:
def readPicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)#cv2.IMREAD_UNCHANGED is important that the output is (x,y,ChannelRGB)
    return(img)

In [4]:
def savePic(picture,fileName,extention,outPath):
    outPath = outPath+fileName+'.'+extention # combines the path with the name and extention of the file
    print(outPath)
    try:
        #imageio.imwrite(outPath,picture,format=extention)# old way
        cv2.imwrite(outPath,picture)#saves Pictures 
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(') #writes an error
        print('--------------------')

In [4]:
def YUVtoRGB(img):
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2BGR, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    #pictureYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV, cv2.IMREAD_UNCHANGED)
    return pictureRGB

In [6]:
#toDo Add parser with args
###### Imput section
print('Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png')
print('FILENAME MUST HAVE the format: 23-y_pred !')
path = input('Path to pictures who should be converted defaut: ./yuvPic/: ') or './yuvPic/'
inputextention = input('What fileextention do the to read pictures have? [default: png]') or 'png'
outputextention = input('Please type outputextention[default: hdr]: ') or 'hdr'
outputpath = input('Where to write the stiched pictures to? [default: ./hdrOut/]: ') or './hdrOut/'
namePic = input('What should be the name of the stiched pictures? [default: pred]') or 'predictedPic'
wantYUV = input('Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]') or 'no'
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention

Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png
FILENAME MUST HAVE the format: 23-y_pred !
Path to pictures who should be converted defaut: ./yuvPic/: /Users/littledragon/Documents/BA 13022020/Daten/test/JSI-GAN_x2_exp1/
What fileextention do the to read pictures have? [default: png]
Please type outputextention[default: hdr]: 
Where to write the stiched pictures to? [default: ./hdrOut/]: 
What should be the name of the stiched pictures? [default: pred]
Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/littledragon/Documents/BA 13022020/Daten/test/JSI-GAN_x2_exp1/'

In [16]:
#Working Version 10022021
start_time = time.time() #start Timer
#TO DO Parser
#desc ='yuv to RGB'
#parser = argparse.ArgumentParser(description=desc)
#parser.add_argument('--yuv', type=str, default='./', help='path to Folder of yuv images')
#print(parser.parse_args())
#what, b = parser.parse_known_args()

#if what.yuv == './' :
 #   print('yes')
start_time = time.time()
i = 0
print(aOp)
while (i <= aOp-1 ): # read y
    print(i)
    try:
        if ((str(os.listdir(path)[i]).split('-')[1]).split('_')[0]) == 'y': # only searching for y picitures
            name = os.listdir(path)[i] #finding the Name
            print(name +' should be the Y')
            picpath = path + name #combining Name and path
            
            #####Y
            picy = readPicture(picpath) #reads a pic y to find the x,y axes should be the same for all pictures
            yuvPic = np.zeros((int(picy.shape[0]),int(picy.shape[1]),3)) # generates the x and y achses and channels of picture
            yuvPic[:,:,0] = picy # packs the Y in pos 1
            num = (str(os.listdir(path)[i]).split('-')[0])#.split('_')[0]
            newPic = path + num + '-u_'+name.split('_')[1]
            
            #####U
            picu = readPicture(newPic)#reads a picture with Ending U
            yuvPic[:,:,1] = picu # packs the u in pos 0
            newPic = path + num + '-v_'+name.split('_')[1] #Generates the Name for v
            
            #####V
            picv = readPicture(newPic)#reads the new picture with ending V
            yuvPic[:,:,2] = picv # packs the u in pos 2
            #Right for the Testdata ist: 0y2u1v <<<<<<<<<<<<<

            if(wantYUV != 'y'):
                
                yuvPic_hdr = YUVtoRGB((yuvPic/((2**10)-1)).astype(np.float32)) #changes the uint16 into float32 and from YUV to RGB
                savePic(((yuvPic_hdr).astype(np.float32)),(str(i)+namePic),outputextention,outputpath) #saves final RGB pic
                
                helpPath = outputpath+'rgb/'
                if not os.path.exists(helpPath):
                    os.mkdir(helpPath)
                rgbPNG= np.clip((yuvPic_hdr*((2**16)-1)), 0, ((2**16)-1)).astype(np.uint16)
                number = int(os.listdir(path)[i].split('-')[0])
                savePic(rgbPNG,(str(number).zfill(6)),'png',helpPath)
                
            if(wantYUV == 'y'):
                yuvPic = YUVtoRGB(yuvPic)
                savePic(yuvPic,(str(i)+namePic),outputextention,outputpath)#saves final YUV pic
    except:
        print('')
    i = i + 1

print("--- %s seconds ---" % (time.time() - start_time))
print(str((time.time() - start_time)/60))
print('------------------------- Done --------------------')

63
0
1
2
3
4
5
11-y_pred.png should be the Y
./hdrOut/5predictedPic.hdr
./hdrOut/rgb/000011.png
6
7
08-y_pred.png should be the Y
./hdrOut/7predictedPic.hdr
./hdrOut/rgb/000008.png
8
9
14-y_pred.png should be the Y
./hdrOut/9predictedPic.hdr
./hdrOut/rgb/000014.png
10
11
12
05-y_pred.png should be the Y
./hdrOut/12predictedPic.hdr
./hdrOut/rgb/000005.png
13
14
19-y_pred.png should be the Y
./hdrOut/14predictedPic.hdr
./hdrOut/rgb/000019.png
15
16
17
18
19
02-y_pred.png should be the Y
./hdrOut/19predictedPic.hdr
./hdrOut/rgb/000002.png
20
13-y_pred.png should be the Y
./hdrOut/20predictedPic.hdr
./hdrOut/rgb/000013.png
21
22
23
24
25
21-y_pred.png should be the Y
./hdrOut/25predictedPic.hdr
./hdrOut/rgb/000021.png
26
16-y_pred.png should be the Y
./hdrOut/26predictedPic.hdr
./hdrOut/rgb/000016.png
27
28
29
30
07-y_pred.png should be the Y
./hdrOut/30predictedPic.hdr
./hdrOut/rgb/000007.png
31
09-y_pred.png should be the Y
./hdrOut/31predictedPic.hdr
./hdrOut/rgb/000009.png
32
33
15-y_p

In [ ]:
#yuvPic

In [ ]:
#rgbPic = YUVtoRGB(())

In [ ]:
#os.listdir(path)[i].split('-')[0]